In [1]:
import os, sys
sys.path.append("..")

In [4]:
!python ../setup.py build 
!python ../setup.py build_ext --inplace

running build
running build_py


error: package directory 'opttrot' does not exist


running build_ext
copying build\lib.win-amd64-cpython-311\pauli_c.cp311-win_amd64.pyd -> opttrot


error: could not create 'opttrot\pauli_c.cp311-win_amd64.pyd': No such file or directory


## PauliElement 

In [1]:
#import os, sys
#sys.path.append("..")

In [1]:
!pip install -e ..

Obtaining file:///C:/Users/cmat411a/Documents/GitHub/Org_OptTrot/OptTrot
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: opttrot
    Found existing installation: opttrot 0.1
    Uninstalling opttrot-0.1:
      Successfully uninstalled opttrot-0.1
  Running setup.py develop for opttrot


In [1]:
!pip list

Package             Version  Editable project location
------------------- -------- --------------------------------------------------------------
appdirs             1.4.4
asttokens           2.4.1
autograd            1.6.2
autoray             0.6.9
cachetools          5.3.3
certifi             2024.2.2
charset-normalizer  3.3.2
colorama            0.4.6
comm                0.2.2
contourpy           1.2.1
cycler              0.12.1
debugpy             1.6.7
decorator           5.1.1
dill                0.3.8
exceptiongroup      1.2.0
executing           2.0.1
fonttools           4.51.0
future              1.0.0
idna                3.6
importlib_metadata  7.1.0
ipykernel           6.29.3
ipython             8.22.2
ipywidgets          8.1.2
jedi                0.19.1
jupyter_client      8.6.1
jupyter_core        5.7.2
jupyterlab_widgets  3.0.10
kiwisolver          1.4.5
llvmlite            0.42.0
matplotlib          3.8.4
matplotlib-inline   0.1.6
mpmath              1.3.0
nest_asyncio 

In [2]:
from opttrot.pauli import PauliElement

ModuleNotFoundError: No module named 'opttrot'

In [6]:
n = 10
p00 = PauliElement(nx=3, nz= 6, n=n, weight=0.124, f=0, set_phase=False)
p01 = PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)


In [7]:
p00

PauliElement(n=10, weight=0.124000, IIIIIIIZYX)

In [8]:
print(p00)

IIIIIIIZYX


## Speed comparsion

In [9]:
from numbers import Number
from functools import reduce
import numpy as np

## Compare with Qiskit

In [10]:
def to_qiskit(self):
    from qiskit.quantum_info import Pauli as qiskit_Pauli
    x_array = np.flip(np.array([int(b) for b in format(self.nx, f"0{self.n}b")], dtype=np.uint))
    z_array = np.flip(np.array([int(b) for b in format(self.nz, f"0{self.n}b")], dtype=np.uint))
    return self.weight, qiskit_Pauli((z_array, x_array))

In [11]:
p00 = PauliElement(nx=3, nz= 6, n=n, weight=0.124, f=0, set_phase=False)
p01 = PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)
w00, q_p00 = to_qiskit(p00)
w01, q_p01 = to_qiskit(p01)


In [12]:
q_p00.compose(q_p01)

Pauli('IIIIIIIXYZ')

In [13]:
p02= p00@p01
p02.pstr

'IIIIIIIXYZ'

## Speed

### Qiskit

In [14]:
%%timeit
q_p00.compose(q_p01)

28.4 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### OptTrot

In [15]:
%%timeit
p00@p01

1.36 µs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Numpy binary

In [16]:
import numpy as np

nx1= np.random.randint(0, 2, size=n, dtype=bool)
nz1= np.random.randint(0, 2, size=n, dtype=bool)
nx2= np.random.randint(0, 2, size=n, dtype=bool)
nz2= np.random.randint(0, 2, size=n, dtype=bool)
f1 = 3
f2 = 4


In [17]:
%%timeit
nx3 = np.bitwise_xor(nx1, nx2)
nz3 = np.bitwise_xor(nz1, nz2)
f3 = f1+f2 + 2*np.bitwise_and(nx1, nz2).astype(int).sum() -  np.bitwise_and(nx3, nz3).astype(int).sum()

11.4 µs ± 68.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Result

- OS: Windows
- CPU: Ryzen 5 
- Python: 3.11.8


|Qiskit| OptTrot| Nb|
|:--:|:--:|:--:|
|27.9 µs ± 1.38 µs|1.35 µs ± 25.2 ns|11.4 µs ± 102 ns|

## PauliPoly

In [19]:
from opttrot.pauli import PauliElement, PauliPoly

In [20]:
p02 =p00@p01
PauliPoly([p00, p01, p02])

PauliPoly(terms:3)[
PauliElement(n=10, weight=0.124000, IIIIIIIZYX),
PauliElement(n=10, weight=0.124000, IIIIIIIYIY),
PauliElement(n=10, weight=0.015376, IIIIIIIXYZ)
]

In [31]:
hash(0.000000000000000000000001)

1528977390404266531

In [21]:
set([p00, p01, p02])

TypeError: unhashable type: 'pauli.PauliElement'